# Téléchargement et installation des libraries

### Clonage du dépot

URL du dépot : https://github.com/tensorflow/models

In [ ]:
# Installation du dépot Tensorflow Models
!git clone https://github.com/tensorflow/models.git

Le modèle que nous allons utiliser est un modèle de reconnaissance d'objet et est nommé **ssd_mobilenet_V2**.

Le tableau à cette adresse récapitule les informations des différents modèles disponnibles. Il permet également d'obtenir les liens pour les télécharger : https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

### Installation des librairies Python

L'adresse suivante exlique comment installer les librairies Python pour utiliser les modèles : https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2.md

In [ ]:
%cd models/research

# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python3 -m pip install .

# Téléchargement du modèle et configuration de l'environnement

### Téléchargement du modèle

In [ ]:
%cd ../..

In [ ]:
# Téléchargement du modèle ssd_mobilenet_v2

!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!tar -zxf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!rm ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

Après le téléchargement, on exporte le modèle en utilisant les propriétés du fichier de configuration `pipeline.config` et en utlisant les fichiers `checkpoint`. Pour cela on utilise le programme `TensorFlow Object Detection Exporter`. 

### Configuration des variables d'environnement

In [ ]:
!echo $PYTHONPATH

In [ ]:
!pwd

In [ ]:
%set_env PYTHONPATH="/env/python:/home/alexandre/Jetbot/Notebooks/Suivi_Objet/models/research:/home/alexandre/Jetbot/Notebooks/Suivi_Objet/models/research/slim:/home/alexandre/Jetbot/Notebooks/Suivi_Objet/models"

In [ ]:
!mkdir tfmodel_ssd_mobilenet_v2_320x320_coco17_tpu-8

### Installation des packages supplémentaires

In [ ]:
!pip3 install tf_slim

### Editer le fichier : deepmac_meta_arch.py
Commenter les lignes suivantes :

    if tf_version.is_tf2():
    import tensorflow_io as tfio

### Copier le répertoire models/official dans /usr/local/lib/python3.6/dist-packages

Entrer la commande suivante dans un terminal:

    sudo cp -r models/official /usr/local/lib/python3.6/dist-packages

# Création du modèle pré-entrainé

In [ ]:
!python3 models/research/object_detection/exporter_main_v2.py --input_type float_image_tensor --trained_checkpoint_dir ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ --pipeline_config_path ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config --output_directory tfmodel_ssd_mobilenet_v2_320x320_coco17_tpu-8

# Utilisation du modèle

Le modèle est maintenant prêt à être utilisé. Le script ci-dessous télécharge une image et effectue un pré-traitement sur celle-ci. Ensuite, elle est envoyée dans le modèle SSD MobileNet V2 de TensorFlow, puis un inférence est lancée. 

### Informations sur le modèle

L'entrée du modèle est une image de dimension 320x320 au format [1,320,320,3] avec des valeurs codées en UINT8.

In [ ]:
!saved_model_cli show --dir tfmodel_ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model --all

### Chargement du modèle

In [ ]:
import tensorflow as tf

model_path = "tfmodel_ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model"
model = tf.saved_model.load(model_path)

### Chargement d'une image dans le tenseur d'entrée

In [ ]:
from PIL import Image
import numpy as np
from io import BytesIO

img_data = tf.io.gfile.GFile('models/research/object_detection/test_images/image2.jpg', 'rb').read()
image = Image.open(BytesIO(img_data))
(im_width, im_height) = image.size
image_np  = np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
input_tensor = np.expand_dims(image_np, 0)

In [ ]:
image

### Traitement de l'image avec le modèle

In [ ]:
detections = model(input_tensor)

### Structure de la sortie

La sortie est un dictionnaire qui contient les informations suivantes :
- `num_detections` : Tenseur au format [N] (type tf.int) qui contient le nombre de détections.
- `detection_boxes` : Tenseur au format [N,4] (type tf.float32) qui contient les coordonnées des boites au format [ymin, xmin, ymax, xmax].
- `detection_classes` : Tenseur au format [N] (type tf.int) qui contient l'index des classes contenues dans le fichier des labels.
- `detection_scores` : Tenseur au format [N] (type tf.float32) qui contient les scores de détection.
- `raw_detection_boxes` : Tenseur au format [1,M,4] (type tf.float32) qui contient les coordonnées des boites sans supressions Non-Max.
- `raw_detection_scores` : Tenseur au format [1,M,91] (type tf.float32) qui contient les scores des classes au format logits des boites précédentes.
- `detection_anchor_indices` : Tenseur au format [N] (type tf.float32) qui contient les indices des ancres des détections après suppression Non-Max.
- `detection_multiclass_scores` : Tenseur au format [1,N,91] qui contient les scores des classes (avec le fond) des boites de détections.

In [ ]:
detections

In [ ]:
detections['detection_boxes']

In [ ]:
detections['detection_boxes'][0,0,:].numpy()

### Affichage des résultats

Dessine les boites sur les trois premiers objets détectés :

In [ ]:
from PIL import ImageDraw
from io import BytesIO


def draw_bbox_and_label_in_image(img, boxes, num_detections, box_width=3):
    """
    Draw bounding boxes and class labels in images
    :param img: PIL Image or np arrays
    :param boxes: in size [num_detections, 4], contains xys or boxes
    :param box_width: the width of boxes
    :return: Image
    """

    draw = ImageDraw.Draw(img)
    width, height = img.size

    for i in range(num_detections):
        ymin, xmin, ymax, xmax = boxes[i]

        ymin = int(ymin * height)
        ymax = int(ymax * height)
        xmin = int(xmin * width)
        xmax = int(xmax * width)

        class_color = "LimeGreen"

        draw.line([(xmin, ymin), (xmax, ymin), (xmax, ymax), (xmin, ymax), (xmin, ymin)], width=box_width, fill=class_color)

    return img

draw_image = draw_bbox_and_label_in_image(image, detections['detection_boxes'][0,0:3,:],3)
draw_image

In [ ]:
print ("Classe du 1er objet : %d (score : %)" %(detections['detection_classes'][0,0]))
print ("Classe du 2eme objet : %d" %(detections['detection_classes'][0,1]))
print ("Classe du 3eme objet : %d" %(detections['detection_classes'][0,2]))